# Ticket Triage with Topic Modelling using Bertopic

In [1]:
from src.dataset import load_dataset

dataset, queue_labels, id2label, label2id, class_weight_dict = load_dataset()
train_dataset, test_dataset = dataset["train"], dataset["test"]

c:\Users\afons\coding-projects\bertopic-ticket-triage\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Casting to class labels: 100%|██████████| 16338/16338 [00:00<00:00, 732590.75 examples/s]
[17:35:50] {c:\Users\afons\coding-projects\bertopic-ticket-triage\src\dataset.py:42} INFO -                                                     text  labels
0      Account Disruption Dear Customer Support Team,...       0
1      Query About Smart Home System Integration Feat...       1
2      Inquiry Regarding Invoice Details Dear Custome...       2
3      Question About Marketing Agency Software Compa...       3
4      Feature Query Dear Customer Support,\n\nI hope...       0
...                                                  ...     ...
16333  Problem with Billing Adjustment An unexpected ...       2
16334  Urgent: 

In [2]:
from src.model import train_supervised_classifier

docs, y = train_dataset["text"], train_dataset["labels"]
base_model = train_supervised_classifier(docs=docs, y=y, class_weight_dict=class_weight_dict)

[17:31:30] {c:\Users\afons\coding-projects\bertopic-ticket-triage\.venv\Lib\site-packages\sentence_transformers\SentenceTransformer.py:219} INFO - Use pytorch device_name: cpu
[17:31:30] {c:\Users\afons\coding-projects\bertopic-ticket-triage\.venv\Lib\site-packages\sentence_transformers\SentenceTransformer.py:227} INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [4]:
test = test_dataset[0]["text"]
test

'Delays in Financial Data Processing The financial organization encountered delays in processing data. The potential reason might be server overload. Efforts to resolve this included rebooting servers and clearing caches, but the problem continues.'

In [ ]:
topic, _ = base_model.transform(test)
topic

[np.int64(0)]

In [ ]:
base_model.get_topic(topic[0])
pred = id2label[topic[0]]
original = id2label[test_dataset[0]["labels"]]
pred, original

('Technical Support', 'IT Support')

In [3]:
def predict(row):
    topic, _ = base_model.transform(row["text"])
    return topic[0]


test_dataset = test_dataset.map(lambda x: {
    "preds" : predict(x)
})

## Baseline is 29.11%
accuracy = test_dataset.filter(lambda x: x["preds"] == x["labels"]).num_rows * 100 / test_dataset.num_rows
accuracy


Filter: 100%|██████████| 4085/4085 [00:00<00:00, 348565.39 examples/s]


29.106487148102815

In [2]:
from src.model import train_zshot_model

zshot_model = train_zshot_model(docs=train_dataset["text"], zeroshot_topic_list=queue_labels)

[17:36:00] {c:\Users\afons\coding-projects\bertopic-ticket-triage\.venv\Lib\site-packages\sentence_transformers\SentenceTransformer.py:219} INFO - Use pytorch device_name: cpu
[17:36:00] {c:\Users\afons\coding-projects\bertopic-ticket-triage\.venv\Lib\site-packages\sentence_transformers\SentenceTransformer.py:227} INFO - Load pretrained SentenceTransformer: ibm-granite/granite-embedding-small-english-r2
Batches: 100%|██████████| 1/1 [00:00<00:00, 19.01it/s]


In [3]:
def predict(row):
    topic, _ = zshot_model.transform(row["text"])
    return topic[0]


test_dataset = test_dataset.map(lambda x: {
    "preds" : predict(x)
})

accuracy = test_dataset.filter(lambda x: x["preds"] == x["labels"]).num_rows * 100 / test_dataset.num_rows
accuracy


Filter: 100%|██████████| 4085/4085 [00:00<00:00, 184428.02 examples/s]


4.063647490820073